In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.regressionplots import influence_plot
import statsmodels.formula.api as smf
import numpy as np
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("bank-full.csv",";")
df.head(8)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no


In [3]:
df.shape

(45211, 17)

In [4]:
df.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [5]:
df["y"].value_counts()

no     39922
yes     5289
Name: y, dtype: int64

In [6]:
df.y.replace(to_replace=['no','yes'],value=[0,1],inplace=True)
df.head(2)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,0


In [7]:
b=df[['y','job','marital','education','default','balance','housing','loan','contact','day','month','duration','campaign','pdays','previous','poutcome','age']]


In [8]:
df=pd.get_dummies(b,columns=['job','marital','education','default','housing','loan','contact','month','poutcome'])
df

,y,balance,day,duration,campaign,pdays,previous,age,job_admin.,job_blue-collar,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0,2143,5,261,1,-1,0,58,0,0,...,0,0,1,0,0,0,0,0,0,1
1,0,29,5,151,1,-1,0,44,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,2,5,76,1,-1,0,33,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0,1506,5,92,1,-1,0,47,0,1,...,0,0,1,0,0,0,0,0,0,1
4,0,1,5,198,1,-1,0,33,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,1,825,17,977,3,-1,0,51,0,0,...,0,0,0,1,0,0,0,0,0,1
45207,1,1729,17,456,2,-1,0,71,0,0,...,0,0,0,1,0,0,0,0,0,1
45208,1,5715,17,1127,5,184,3,72,0,0,...,0,0,0,1,0,0,0,0,1,0
45209,0,668,17,508,4,-1,0,57,0,1,...,0,0,0,1,0,0,0,0,0,1


In [9]:
df.columns

Index(['y', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'age', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'default_no', 'default_yes', 'housing_no',
       'housing_yes', 'loan_no', 'loan_yes', 'contact_cellular',
       'contact_telephone', 'contact_unknown', 'month_apr', 'month_aug',
       'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'poutcome_failure', 'poutcome_other', 'poutcome_success',
       'poutcome_unknown'],
      dtype='object')

# Building a logistic regression model

In [10]:
x=df.iloc[:,1:]
y=df.iloc[:,0]

In [11]:
x.head(10)

,balance,day,duration,campaign,pdays,previous,age,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,2143,5,261,1,-1,0,58,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,29,5,151,1,-1,0,44,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,2,5,76,1,-1,0,33,0,0,1,...,0,0,1,0,0,0,0,0,0,1
3,1506,5,92,1,-1,0,47,0,1,0,...,0,0,1,0,0,0,0,0,0,1
4,1,5,198,1,-1,0,33,0,0,0,...,0,0,1,0,0,0,0,0,0,1
5,231,5,139,1,-1,0,35,0,0,0,...,0,0,1,0,0,0,0,0,0,1
6,447,5,217,1,-1,0,28,0,0,0,...,0,0,1,0,0,0,0,0,0,1
7,2,5,380,1,-1,0,42,0,0,1,...,0,0,1,0,0,0,0,0,0,1
8,121,5,50,1,-1,0,58,0,0,0,...,0,0,1,0,0,0,0,0,0,1
9,593,5,55,1,-1,0,43,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [12]:
df.sample(2)

,y,balance,day,duration,campaign,pdays,previous,age,job_admin.,job_blue-collar,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
44986,0,0,11,146,4,-1,0,51,0,0,...,0,0,0,0,1,0,0,0,0,1
1878,0,420,9,76,1,-1,0,56,0,1,...,0,0,1,0,0,0,0,0,0,1


In [13]:
y.head

<bound method NDFrame.head of 0        0
1        0
2        0
3        0
4        0
        ..
45206    1
45207    1
45208    1
45209    0
45210    0
Name: y, Length: 45211, dtype: int64>

In [14]:
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn import metrics

In [15]:
model=LogisticRegression()
model.fit(x,y)

LogisticRegression()

In [16]:
model.intercept_

array([-0.27847713])

In [17]:
np.round(model.coef_,2)

array([[ 0.  , -0.01,  0.  , -0.38,  0.  , -0.12, -0.02, -0.03, -0.25,
        -0.03, -0.01,  0.01,  0.16, -0.02, -0.09,  0.03, -0.07,  0.  ,
         0.  ,  0.02, -0.17, -0.12, -0.07, -0.27,  0.05,  0.01, -0.26,
        -0.02,  0.36, -0.64, -0.08, -0.2 ,  0.09,  0.05, -0.42,  0.04,
         0.02,  0.03, -0.01, -0.01, -0.08, -0.06,  0.09, -0.4 , -0.06,
         0.1 ,  0.07, -0.13, -0.03,  0.31, -0.42]])

In [18]:
y_pred=model.predict(x)

# Creating a dataframe

In [19]:
y_pred_df=pd.DataFrame({'actual':y,'predicted_values':model.predict(x)})

In [20]:
y_pred_df

,actual,predicted_values
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
45206,1,1
45207,1,0
45208,1,1
45209,0,0


In [21]:
y_pred_df["actual"].value_counts()

0    39922
1     5289
Name: actual, dtype: int64

In [22]:
y_pred_df["predicted_values"].value_counts()

0    43229
1     1982
Name: predicted_values, dtype: int64

In [23]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score as ac


In [24]:
confusion_matrix=confusion_matrix(y,y_pred)
print(confusion_matrix)

[[39130   792]
 [ 4099  1190]]


In [25]:
ac(y,y_pred)

0.8918183627878171

# classification report

In [26]:
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94     39922
           1       0.60      0.22      0.33      5289

    accuracy                           0.89     45211
   macro avg       0.75      0.60      0.63     45211
weighted avg       0.87      0.89      0.87     45211



# Predicting using Train and Test model

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=12)

In [29]:
model1=LogisticRegression()
model1.fit(x_train,y_train)
y_pred=model1.predict(x_test)

In [30]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94     11990
           1       0.60      0.24      0.34      1574

    accuracy                           0.89     13564
   macro avg       0.75      0.61      0.64     13564
weighted avg       0.87      0.89      0.87     13564



# Using Decision tree classifier

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [32]:

model2=DecisionTreeClassifier(criterion='gini',min_samples_split=5,max_depth=10)
model2.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=10, min_samples_split=5)

In [34]:
y_pred=model2.predict(x_test)

In [35]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94     11990
           1       0.61      0.41      0.49      1574

    accuracy                           0.90     13564
   macro avg       0.77      0.69      0.72     13564
weighted avg       0.89      0.90      0.89     13564

